<a href="https://colab.research.google.com/github/athishr88/NN_DL/blob/main/CT9/CT9.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install transformers

     |████████████████████████████████| 3.8 MB 5.4 MB/s 
     |████████████████████████████████| 596 kB 23.8 MB/s 
     |████████████████████████████████| 6.5 MB 25.6 MB/s 
     |████████████████████████████████| 67 kB 4.7 MB/s 
     |████████████████████████████████| 895 kB 37.9 MB/s 
  Attempting uninstall: pyyaml
    Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully uninstalled PyYAML-3.13


In [2]:
# Part 1: BERT model with pipeline method

from transformers import pipeline

classifier = pipeline("sentiment-analysis")

No model was supplied, defaulted to distilbert-base-uncased-finetuned-sst-2-english (https://huggingface.co/distilbert-base-uncased-finetuned-sst-2-english)


Downloading:   0%|          | 0.00/629 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/255M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/226k [00:00<?, ?B/s]

In [3]:
classifier("I am sad todas")

[{'label': 'NEGATIVE', 'score': 0.9977282881736755}]

In [4]:
classifier = pipeline("sentiment-analysis", model="nlptown/bert-base-multilingual-uncased-sentiment")

Downloading:   0%|          | 0.00/953 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/638M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/39.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/851k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [5]:
classifier("Big dissapointment")

[{'label': '1 star', 'score': 0.3411187529563904}]

In [6]:
#Part 2: BERT with direct use of models
!pip install datasets

     |████████████████████████████████| 325 kB 5.3 MB/s 
     |████████████████████████████████| 212 kB 46.0 MB/s 
     |████████████████████████████████| 1.1 MB 44.6 MB/s 
     |████████████████████████████████| 134 kB 45.7 MB/s 
     |████████████████████████████████| 127 kB 32.8 MB/s 
     |████████████████████████████████| 94 kB 1.4 MB/s 
     |████████████████████████████████| 144 kB 46.8 MB/s 
     |████████████████████████████████| 271 kB 43.3 MB/s 
  Attempting uninstall: urllib3
    Found existing installation: urllib3 1.24.3
    Uninstalling urllib3-1.24.3:
      Successfully uninstalled urllib3-1.24.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
datascience 0.10.6 requires folium==0.2.1, but you have folium 0.8.3 which is incompatible.


In [7]:
from datasets import load_dataset

dataset = load_dataset("emotion")

Using custom data configuration default


Generating train split:   0%|          | 0/16000 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Dataset emotion downloaded and prepared to /root/.cache/huggingface/datasets/emotion/default/0.0.0/348f63ca8e27b3713b6c04d723efe6d824a56fb3d1449794716c0f0296072705. Subsequent calls will reuse this data.


  0%|          | 0/3 [00:00<?, ?it/s]

In [8]:
# Tokenize dataset

from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("bert-base-cased")

def tokenize_function(examples):
  return tokenizer(examples['text'], padding='max_length', truncation=True)

Downloading:   0%|          | 0.00/29.0 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/208k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/426k [00:00<?, ?B/s]

In [9]:
tokenized_dataset = dataset.map(tokenize_function, batched=True)

  0%|          | 0/16 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

  0%|          | 0/2 [00:00<?, ?ba/s]

In [10]:
# Choose a small subset of train and val
small_train_dataset = tokenized_dataset['train'].shuffle(seed=42).select(range(1000))
small_eval_dataset = tokenized_dataset['test'].shuffle(seed=42).select(range(1000))

In [11]:
# Convert tokenized dataset to tensorflow dataset
# DefaultDataCollator

from transformers import DefaultDataCollator
data_collator = DefaultDataCollator(return_tensors='tf')

tf_train_dataset = small_train_dataset.to_tf_dataset(
    columns = ["attention_mask", "input_ids", "token_type_ids"],
    label_cols = ["labels"],
    shuffle = True,
    collate_fn = data_collator,
    batch_size = 8
)

tf_validation_dataset = small_eval_dataset.to_tf_dataset(
    columns = ["attention_mask", "input_ids", "token_type_ids"],
    label_cols = ["labels"],
    shuffle = False,
    collate_fn = data_collator,
    batch_size = 8
)

In [12]:
from transformers import TFAutoModelForSequenceClassification

model = TFAutoModelForSequenceClassification.from_pretrained("bert-base-cased", num_labels=6)

# Compile and train
import tensorflow as tf

model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=5e-5),
              loss = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
              metrics = tf.metrics.SparseCategoricalAccuracy())


Downloading:   0%|          | 0.00/502M [00:00<?, ?B/s]

All model checkpoint layers were used when initializing TFBertForSequenceClassification.

Some layers of TFBertForSequenceClassification were not initialized from the model checkpoint at bert-base-cased and are newly initialized: ['classifier']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [13]:
model.fit(tf_train_dataset, validation_data=tf_validation_dataset, epochs=3)

Epoch 1/3
125/125 [==============================] - 329s 2s/step - loss: 1.3655 - sparse_categorical_accuracy: 0.5030 - val_loss: 0.8527 - val_sparse_categorical_accuracy: 0.7560
Epoch 2/3
125/125 [==============================] - 302s 2s/step - loss: 0.7089 - sparse_categorical_accuracy: 0.7690 - val_loss: 0.6074 - val_sparse_categorical_accuracy: 0.8080
Epoch 3/3
125/125 [==============================] - 302s 2s/step - loss: 0.3350 - sparse_categorical_accuracy: 0.9000 - val_loss: 0.5238 - val_sparse_categorical_accuracy: 0.8490


In [14]:
model.evaluate(tf_validation_dataset)

125/125 [==============================] - 79s 635ms/step - loss: 0.5238 - sparse_categorical_accuracy: 0.8490


[0.5238471031188965, 0.8489999771118164]

In [16]:
## Part 3
# Generate text GPT-v1 model with pipeline

model_generate = pipeline("text-generation", model='openai-gpt')
model_generate("Today its too cold")

Some weights of OpenAIGPTLMHeadModel were not initialized from the model checkpoint at openai-gpt and are newly initialized: ['lm_head.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


[{'generated_text': 'Today its too cold to run the water in the kitchen sink, he opened the window, walked to the top of the driveway and gazed out over the fields. for a moment he was struck by how clear everything was, with nothing to block the view'}]

In [17]:
## Part 4
# GPT-1 with direct use of the model
# Download the dataset
!wget https://github.com/GeorgeDittmar/reddit-scan-datasets/blob/master/data_sets/investing/submission_titles_investing_100k.txt

--2022-03-20 22:50:36--  https://github.com/GeorgeDittmar/reddit-scan-datasets/blob/master/data_sets/investing/submission_titles_investing_100k.txt
Resolving github.com (github.com)... 140.82.114.4
Connecting to github.com (github.com)|140.82.114.4|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: unspecified [text/html]
Saving to: ‘submission_titles_investing_100k.txt’

submission_titles_i     [ <=>                ] 124.29K  --.-KB/s    in 0.08s   

2022-03-20 22:50:37 (1.61 MB/s) - ‘submission_titles_investing_100k.txt’ saved [127276]



In [28]:
## Read, preprocess and split dataset (train, val)
from sklearn.model_selection import train_test_split

# Add special tokens
with open("submission_titles_investing_100k.txt", 'r') as data:
  dataset = ["<|title|"+ x.strip() for x in data.readlines()]

train, eval = train_test_split(dataset, train_size=0.9, random_state=2020)

with open('train_tmp.txt', 'w') as file_h:
  file_h.write("<endoftext>".join(train))

with open('eval_tmp.txt', 'w') as file_h:
  file_h.write("<endoftext>".join(eval))

In [29]:
## Import tokenizer and model
from transformers import OpenAIGPTTokenizerFast, TFOpenAIGPTLMHeadModel

model = TFOpenAIGPTLMHeadModel.from_pretrained("openai-gpt", from_pt=True)
tokenizer = OpenAIGPTTokenizerFast.from_pretrained("openai-gpt")

Some weights of the PyTorch model were not used when initializing the TF 2.0 model TFOpenAIGPTLMHeadModel: ['h.4.attn.bias', 'h.10.attn.bias', 'h.0.attn.bias', 'h.2.attn.bias', 'h.5.attn.bias', 'h.1.attn.bias', 'h.9.attn.bias', 'h.8.attn.bias', 'h.11.attn.bias', 'h.7.attn.bias', 'h.3.attn.bias', 'h.6.attn.bias']
- This IS expected if you are initializing TFOpenAIGPTLMHeadModel from a PyTorch model trained on another task or with another architecture (e.g. initializing a TFBertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing TFOpenAIGPTLMHeadModel from a PyTorch model that you expect to be exactly identical (e.g. initializing a TFBertForSequenceClassification model from a BertForSequenceClassification model).
All the weights of TFOpenAIGPTLMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFOpenAIGPTLMHeadModel for predicti

In [30]:
## Example input
input_ids = tokenizer.encode("Some text to encode", return_tensors="tf")

In [35]:
generated_text = model.generate(input_ids,
                       max_length=150,
                       num_return_sequences=5,
                       no_repeat_ngram_size=2,
                       repetition_penalty=1.5,
                       top_p=0.92,
                       temperature=0.85,
                       do_sample=True,
                       top_k=125,
                       early_stopping=True)

In [36]:
for i, text in enumerate(generated_text):
  print(f"{i} {tokenizer.decode(text, skip_special_tokens=True)}")
  print()

0 some text to encode the message on her phone. 
 " what is that? are you okay, darling? you look pale. " he took off his jacket and put it over my shoulders before we left the room. when we walked into the living area, i heard a voice behind me say, " oh man, he's hot. that one is going down. now, if only i could have just had the good sense to take him upstairs. maybe then i wouldn't have ended up with him. you know how much i hate when guys do that. just hold his hand as we go in there and make sure nothing happens between us. i am sorry, sweetheart, but this is for your own good. have no worries about anything

1 some text to encode my text message, " i said. 
 " it says i'm not feeling well. i think you should probably take care of this for me, okay? " she nodded and then closed her laptop. after a few minutes, we all started getting ready for bed. abby opened the door at about three in the morning and went back into the kitchen to pack our things and get us out the front door. as